# Analyze 2020 solution

## Goal

Analyze the 2020 solution to see how easy is to create an ensemble with it.

## Imports

In [ ]:
import os
import json
import random
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib as mpl
import numpy as np

plt.plot()
plt.close('all')
plt.rcParams["figure.figsize"] = (15, 4)
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['font.size'] = 16

## Code

In [ ]:
def plot_task(task, task_id):
    all_samples = task['train'] + task['test']
    for plot_idx, sample in enumerate(all_samples):
        plt.subplot(1, len(all_samples), plot_idx+1)
        plot_grid(sample['input'])
        if plot_idx < len(task['train']):
            plt.title(f'train {plot_idx}')
        else:
            plt.title(f'test {plot_idx-len(task["train"])}')
    plt.suptitle(f'Inputs for task {task_id}')
    plt.show()
    for plot_idx, sample in enumerate(all_samples):
        plt.subplot(1, len(all_samples), plot_idx+1)
        plot_grid(sample['output'])
        if plot_idx < len(task['train']):
            plt.title(f'train {plot_idx}')
        else:
            plt.title(f'test {plot_idx-len(task["train"])}')
    plt.suptitle(f'Outputs for task {task_id}')
    plt.show()


def plot_grid(grid):
    grid = np.array(grid, dtype=int)
    cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
    norm = colors.Normalize(vmin=0, vmax=9)
    plt.imshow(grid, cmap=cmap, norm=norm)
    plt.grid(True,which='both',color='lightgrey', linewidth=0.5)
    plt.xticks(np.arange(-0.5, grid.shape[1]), [])
    plt.yticks(np.arange(-0.5, grid.shape[0]), [])
    plt.xlim(-0.5, grid.shape[1]-0.5)

    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            plt.text(j, i, grid[i, j], ha='center', va='center')

In [ ]:
def plot_solution(solution, title):
    for plot_idx, sample in enumerate(solution.values()):
        plt.subplot(1, len(solution), plot_idx+1)
        plot_grid(sample)
        plt.title(f'attempt {plot_idx + 1}')
    plt.suptitle(title)
    plt.show()

## Analysis

In [ ]:
with open('/mnt/hdd0/Kaggle/arc24/submissions/submission_2020_solution.json', 'r') as f:
    data = json.load(f)

In [ ]:
len(data)

In [ ]:
np.unique([len(values[0]) for values in data.values()], return_counts=True)

It seems that all tasks received two attempts, but are they different?

In [ ]:
for task_id, values in data.items():
    for sample_id, sample in enumerate(values):
        if sample['attempt_1'] == sample['attempt_2']:
            print(task_id, sample_id)

Only one of the submissions has repeated the same grid. It seems that the ensemble is not going to be trivial.

## Checking the accuracy of the attempts

In [ ]:
with open('/mnt/hdd0/Kaggle/arc24/data/arc-agi_training_solutions.json', 'r') as f:
    solutions = json.load(f)

In [ ]:
def measure_accuracy(submission, solution):
    accuracy = dict(attempt_1=0, attempt_2=0)
    for task_id, values in submission.items():
        for sample_id, sample in enumerate(values):
            for key in accuracy.keys():
                if sample[key] == solution[task_id][sample_id]:
                    accuracy[key] += 1

    return accuracy

In [ ]:
measure_accuracy(data, solutions)

The first attempt is right 50 times, the second is only 9. So we could try to make a submission with just the first attempt.

That is not optimal, but to make it better I should understand how the solution is generated.